In [1]:


###先进行之前的emb_path的训练
#Options
addRawFeat = True
base_path = ''
feature_networks_integration = [ 'path','cell']
#feature_networks_integration = [ 'exp']
node_networks = [ 'path','cell']
#node_networks = [ 'exp']
int_method = 'MLP' # 'MLP' or 'XGBoost' or 'RF' or 'SVM'
xtimes = 50 
xtimes2 = 10 

feature_selection_per_network = [False]*len(feature_networks_integration)
top_features_per_network = [50, 50]
optional_feat_selection = False
boruta_runs = 100
boruta_top_features = 50

max_epochs = 500
min_epochs = 200
patience = 30
learning_rates = [0.01, 0.001, 0.0001]
#learning_rates = [0.0001]
# hid_sizes = [16, 32, 64, 128, 256, 512] 
hid_sizes = [512] 
random_state = 404

In [2]:
# MOGAT run
print('MOGAT is setting up!')
from lib import function
import time, io
import os, pyreadr, itertools
import pickle as pickle
from sklearn.metrics import f1_score, accuracy_score
import statistics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, RandomizedSearchCV, GridSearchCV
import pandas as pd
import numpy as np
import os
import torch
import argparse
import errno
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


MOGAT is setting up!


In [3]:
if ((True in feature_selection_per_network) or (optional_feat_selection == True)):
    import rpy2
    import rpy2.robjects as robjects
    from rpy2.robjects.packages import importr
    utils = importr('utils')
    rFerns = importr('rFerns')
    Boruta = importr('Boruta')
    pracma = importr('pracma')
    dplyr = importr('dplyr')
    import re

In [4]:
# Parser
parser = argparse.ArgumentParser(description='''An integrative node classification framework, called MOGAT 
(a cancer subtype prediction methodology), that utilizes graph attentions on multiple datatype-specific networks that are annotated with multiomics datasets as node features. 
This framework is model-agnostic and could be applied to any classification problem with properly processed datatypes and networks.
In our work, MOGAT was applied specifically to the breast cancer subtype prediction problem by applying attentions on patient similarity networks
constructed based on multiple biological datasets from breast tumor samples.''')
parser.add_argument('-data', "--data_location", nargs = 1, default = ['data(CRC)/训练'])
parser.add_argument("--verbosity", help="increase output verbosity")
args = parser.parse_args(args=[])
dataset_name = args.data_location[0]

path = base_path  + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cpu')  # 将设备设置为 CPU
torch.set_default_tensor_type('torch.FloatTensor')  # 设置默认张量类型为 CPU 浮点张量

In [5]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

class Data:
    def __init__(self, x=None, y=None):
        self.x=x
        self.y=y

    def cpu(self):
        self.x = self.x.cpu()
        self.y = self.y.cpu()

if ((True in feature_selection_per_network) or (optional_feat_selection == True)):
    import rpy2
    import rpy2.robjects as robjects
    from rpy2.robjects.packages import importr
    utils = importr('utils')
    rFerns = importr('rFerns')
    Boruta = importr('Boruta')
    pracma = importr('pracma')
    dplyr = importr('dplyr')
    import re

In [7]:
path = base_path  + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cpu')
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
#torch.cuda.set_device(6)


data_path_node =  base_path + dataset_name +'/'
run_name = 'MOGAT_'+  dataset_name + '_results_1'
save_path = base_path + run_name + '/'
excel_file = save_path + "MOGAT_results.xlsx"

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

# 读取 CSV 文件
file = base_path + dataset_name + '/labels.csv'  # 假设您的标签文件为 labels.csv
print("Reading:", file)
# 使用 pandas 读取 CSV 文件
labels_df = pd.read_csv(file)
# 假设标签在 CSV 文件中的一列名为 'label'，您可以根据实际情况调整列名
labels = labels_df['diagnosis'].values  # 将标签转换为 NumPy 数组

file = base_path + 'data/' + dataset_name + '/mask_values.pkl'
if os.path.exists(file):
    with open(file, 'rb') as f:
        train_valid_idx, test_idx = pickle.load(f)
else:
    train_valid_idx, test_idx= train_test_split(np.arange(len(labels)), test_size=0.20, shuffle=True, stratify=labels, random_state=random_state)

start = time.time()

is_first = 0

print('MOGAT is running..')


Reading: data(CRC)/训练/labels.csv
MOGAT is running..


In [8]:
addFeatures = []
t = range(len(node_networks))
trial_combs = []
for r in range(1, len(t) + 1):
    trial_combs.extend([list(x) for x in itertools.combinations(t, r)])
new_trial_combs = []
for set1 in trial_combs:
    new_trial_combs.append(list(set1))
trial_combs = new_trial_combs

device = torch.device('cpu')

In [10]:
from sklearn.model_selection import KFold
import joblib  # 用于保存模型
import pandas as pd

# 设定交叉验证参数
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# 创建一个空的 DataFrame 来保存结果
results_df = pd.DataFrame(columns=['Trial', 'Fold', 'True Labels', 'Predicted Labels', 'Sample Indices', 'Accuracy', 'F1 Score'])

for trials in range(len(trial_combs)):
    node_networks2 = [node_networks[i] for i in trial_combs[trials]]
    netw_base = node_networks2[0]
    emb_file = save_path + 'Emb_' + netw_base + '.pkl'
    
    with open(emb_file, 'rb') as f:
        emb = CPU_Unpickler(f).load()
    emb = emb.cpu()
    
    if len(node_networks2) > 1:
        for netw_base in node_networks2[1:]:
            emb_file = save_path + 'Emb_' + netw_base + '.pkl'
            with open(emb_file, 'rb') as f:
               cur_emb = CPU_Unpickler(f).load()
               cur_emb = cur_emb.cpu()
            emb = torch.cat((emb, cur_emb), dim=1)
    
    emb = emb.cpu()
    
    # 处理特征和标签
    # ... (省略特征处理的代码，保持不变) ...
    
    labels_tensor = torch.tensor(labels, device=device)
    data = Data(x=emb, y=labels_tensor)
    data.cpu()
    
    # 创建训练和测试掩码
    train_mask = np.array([i in set(train_valid_idx) for i in range(data.x.shape[0])])
    data.train_mask = torch.tensor(train_mask, device=device)
    test_mask = np.array([i in set(test_idx) for i in range(data.x.shape[0])])
    data.test_mask = torch.tensor(test_mask, device=device)
    
    X = pd.DataFrame(data.x.numpy())
    y = pd.DataFrame(data.y.numpy()).values.ravel()
    
    # 开始五倍交叉验证
    for fold, (train_index, test_index) in enumerate(kf.split(X)):
        X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
        y_train_fold, y_test_fold = y[train_index], y[test_index]
        
        if int_method == 'MLP':
            params = {'hidden_layer_sizes': [ (64, 32)],
                    'learning_rate_init': [0.001],
                    'max_iter': [ 1500],
                    'n_iter_no_change': [100]}
            search = RandomizedSearchCV(estimator = MLPClassifier(solver = 'adam', activation = 'relu', early_stopping = True), 
                                        return_train_score = True, scoring = 'f1_macro', 
                                        param_distributions = params, cv = 4, n_iter = xtimes, verbose = 0)
            search.fit(X_train_fold, y_train_fold)
            model = MLPClassifier(solver = 'adam', activation = 'relu', early_stopping = True,
                                max_iter = search.best_params_['max_iter'], 
                                n_iter_no_change = search.best_params_['n_iter_no_change'],
                                hidden_layer_sizes = search.best_params_['hidden_layer_sizes'],
                                learning_rate_init = search.best_params_['learning_rate_init'])
        elif int_method == 'XGBoost':
            # ... (XGBoost模型的参数设置和训练) ...
            model = XGBClassifier(...)  # 使用最佳参数初始化模型
        elif int_method == 'RF':
            # ... (随机森林模型的参数设置和训练) ...
            model = RandomForestClassifier(...)  # 使用最佳参数初始化模型
        elif int_method == 'SVM':
            # ... (SVM模型的参数设置和训练) ...
            model = SVC(...)  # 使用最佳参数初始化模型
        
        # 训练模型
        model.fit(X_train_fold, y_train_fold)
        
        # 进行预测
        y_pred_fold = model.predict(X_test_fold)
        
        # 计算准确率和 F1 分数
        accuracy = accuracy_score(y_test_fold, y_pred_fold)
        f1 = f1_score(y_test_fold, y_pred_fold, average='macro')
        # 保存结果
        results_to_append = []
        for idx, (true_label, pred_label) in enumerate(zip(y_test_fold, y_pred_fold)):
            results_to_append.append({
                'Trial': trials,
                'Fold': fold,
                'True Labels': true_label,
                'Predicted Labels': pred_label,
                'Sample Indices': test_index[idx],
                'Accuracy': accuracy,
                'F1 Score': f1
            })
        # 使用 pd.DataFrame() 创建新的 DataFrame 并与原 DataFrame 合并
        results_df = pd.concat([results_df, pd.DataFrame(results_to_append)], ignore_index=True)

        # 保存模型
        joblib.dump(model, f'model_trial_{trials}_fold_{fold}.model')



# 保存所有结果到 CSV 文件
results_df.to_csv('data(CRC)\\cross_validation_results.csv', index=False)

print("Models and results saved successfully.")


Models and results saved successfully.


In [11]:
import pandas as pd

# 假设三个 CSV 文件的路径
file1 = 'data(CRC)\\外部测试\\path.csv'
file2 = 'data(CRC)\\外部测试\\cell.csv'
file3 = 'MOGAT_data(CRC)\\训练_results_1\\Output_path.csv'
file4='MOGAT_data(CRC)\\训练_results_1\\Output_cell.csv'
# 读取 CSV 文件
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)
df4 = pd.read_csv(file4)
# 将三个 DataFrame 的列整合到一起
X_test = pd.concat([df3,df4], axis=1)
X_test.columns = range(1024)
# 输出整合后的 X_test 的形状和内容
print(f"X_test shape: {X_test.shape}")
print(X_test.head())

X_test shape: (369, 1024)
       0         1         2         3         4         5         6     \
0 -0.150634 -0.682642  0.485043  0.228134  0.203529 -0.921121 -0.887200   
1 -0.098627 -0.530374  0.321191  0.254006  0.153294 -0.983621 -0.874228   
2  0.201044 -0.805119  0.140224  0.215465  0.238747 -0.923598 -0.912505   
3  0.174804 -0.790710  0.411063  0.088463 -0.267014 -0.933273 -0.874443   
4  0.380464 -0.787200 -0.089935  0.233919  0.023287 -0.978513 -0.901680   

       7         8         9     ...      1014      1015      1016      1017  \
0 -0.815090 -0.939623 -1.018917  ... -0.721703 -1.166818 -0.665491 -0.823111   
1 -0.443499 -0.865848 -1.112709  ... -0.651851 -1.274266 -0.736652 -0.896479   
2 -1.038073 -0.995203 -0.998607  ... -0.793779 -1.209201 -0.926957 -0.919307   
3 -0.936515 -0.952081 -0.984277  ... -0.780357 -1.032351 -0.937825 -0.843275   
4 -0.958313 -0.947101 -0.947222  ... -0.655145 -1.141905 -0.882093 -0.947972   

       1018      1019      1020      1021 

In [12]:
path = base_path  + dataset_name
if not os.path.exists(path):
    raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), path)
        
device = torch.device('cpu')
#torch.set_default_tensor_type('torch.cuda.FloatTensor')
#torch.cuda.set_device(6)


data_path_node =  base_path + dataset_name +'/'
run_name = 'MOGAT_'+  dataset_name + '_results_1'
save_path = base_path + run_name + '/'
excel_file = save_path + "MOGAT_results.xlsx"

if not os.path.exists(base_path + run_name):
    os.makedirs(base_path + run_name + '/')

# 读取 CSV 文件
file = 'data(CRC)\\外部测试\\labels.csv'  # 假设您的标签文件为 labels.csv
print("Reading:", file)
# 使用 pandas 读取 CSV 文件
labels_df = pd.read_csv(file)
# 假设标签在 CSV 文件中的一列名为 'label'，您可以根据实际情况调整列名
labels = labels_df['diagnosis'].values  # 将标签转换为 NumPy 数组

file = base_path + dataset_name + '/mask_values.pkl'

start = time.time()

is_first = 0

print('MOGAT is running..')

Reading: data(CRC)\外部测试\labels.csv
MOGAT is running..


In [33]:
import joblib
# 2. 加载保存的模型
model_filename = 'model_trial_2_fold_4.model'  # 修改最后0，1，2，3，4
model = joblib.load(model_filename)

In [34]:
labels_tensor = torch.tensor(labels, device=device)  # 将 labels 转换为张量

data = Data(x=X_test, y=labels_tensor)  # 使用张量创建 Data 对象

In [35]:
X_test = pd.DataFrame(data.x)
y_test = pd.DataFrame(data.y).values.ravel()

In [36]:

# 使用训练好的模型进行预测
predictions_external = model.predict(X_test)

In [37]:
predictions_external = pd.DataFrame(predictions_external)
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
result_df = pd.concat([labels_df, predictions_external], axis=1)


# 计算准确率和 F1 分数
# 假设 labels_df 的列名为 'True Labels'，而 predictions_external 的列名为 'Predicted Labels'
true_labels = labels_df['diagnosis']  # 替换为真实标签的列名
predicted_labels = predictions_external.iloc[:, 0]  # 替换为预测结果的列名

accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='macro')  # 使用 'macro' 平均方式

# 将结果保存到 CSV 文件
result_df.to_csv('data(CRC)\\\外部测试\\test_results_5.csv', index=False)###修改1，2，3，4，5

# 打印准确率和 F1 分数
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.5203
F1 Score: 0.4909
